# 16 MeV D on Ta 

This notebook documents the steps taken to calibrate, aquire, and analyze data from activation foils used to unfold the neutron spectrum from 16 MeV D breakup on Ta.  

This Notebook is broken into the following main sections: <br\>
1) Experiment Planning  <br\>

In [1]:
import os
import sys
import peakutils

import pandas as pd
import numpy as np
import copy as cp

from math import sqrt, exp, log
from datetime import datetime
from scipy.optimize import curve_fit
from peakutils.plot import plot as pplot
from matplotlib import pyplot
%matplotlib inline

# Path to support functions 
sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/GeneralNuclear'))
from Counting import parse_spe, peak_counts, germanium_eff, foil_count_time, volume_solid_angle, optimal_count_plan
from BasicNuclearCalcs import decay, production_decay, get_decay_const, fractional_solid_angle

## Simulated Results

### MCNP

The experimental configuration was simulated in MCNP to get the expected reaction rates.  The Meulder's deuterons on 197Au was used as the source, and the foil pack was suspended center beam 34.62 cm from the wall.

The results are located at <path>\Dropbox\UCB\Research\ETAs\88Inch\Data\Simulated\Activation\16MeVTa\Model

They are sumarized at:

In [2]:
simPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Simulated/Activation/16MeVTa/16MeVTaFoils.xlsx"

## Experiment Planning

Develop a count plan based on the planned irradiation. 

### Experimental parameters

Specifies experimental parameters for the beam and counting facility used.

runTime = the beam irradiation time in s

transitTime = the minimum time to counting of the first foil in s

background = the background rate at the peak of interest in counts/s

detR = radius of the detector in cm

det2FoilDist = the distance from the detector face to the foil in cm

sigma = the desired counting statistics level

In [80]:
runTime = 3600*24
transitTime = 600
nChannels=7

background=0.01
detR=3.245
det2FoilDist=1

# Same statistics for each channel
#sigma=0.01
#sigma = [sigma]*nChannels

# Variable statistics
sigma=[0.02,0.02,0.1]

### Read in Simulation data

Read from the designated file path.  Manipulate the file to build a dataframe with the information needed for the count plan.  

In [81]:
simFoilParams = pd.read_excel(simPath)

# Delete unneccesary columns for readability and make the index the reactant
simFoilParams.index=simFoilParams.Rx
del simFoilParams['Rx']
del simFoilParams['Thickness [cm]']
del simFoilParams['Density']
del simFoilParams['AW']
del simFoilParams['Lambda [s^-1]']

# Rename columns for ease of access and add in the statistics column
simFoilParams.columns=['foil','product','gammaEnergy','br','normalization','rxRate','rxRateSigma','foilR','weightFrac',\
                       'volume', 'halfLife']
simFoilParams['relStat']=sigma

# Put branching ratios in fractional form
simFoilParams['br']=simFoilParams['br']/100.

# Calculate the activity following transit time decay for each rx; delete columns no longer needed
simFoilParams['initActivity']=0.0
simFoilParams['activityUncert']=0.0
for ind in simFoilParams.index:
    simFoilParams.at[ind,'initActivity']=production_decay(simFoilParams.at[ind,'halfLife'], 0, runTime, \
                                                    simFoilParams.at[ind,'rxRate'], \
                                                    simFoilParams.at[ind,'normalization'], \
                                                    simFoilParams.at[ind,'volume'], transitTime)\
                                                    *get_decay_const(simFoilParams.at[ind,'halfLife'])\
                                                    *simFoilParams.at[ind,'br']
    simFoilParams.at[ind,'activityUncert']=simFoilParams.at[ind,'initActivity']*simFoilParams.at[ind,'rxRateSigma']
del simFoilParams['rxRate']
del simFoilParams['normalization']
del simFoilParams['volume']
del simFoilParams['rxRateSigma']

print simFoilParams

            foil product  gammaEnergy      br  foilR  weightFrac   halfLife  \
Rx                                                                            
58Ni(n,p)    NiP    Co58       810.76  0.9945    2.5    0.680769  6122304.0   
115In(n,n')   In  In115M       335.00  0.4580    2.5    0.957100    16200.0   
115In(n,g)    In  In116M      1293.56  0.8480    2.5    0.957100     3247.2   

             relStat  initActivity  activityUncert  
Rx                                                  
58Ni(n,p)       0.02      0.424915        0.000425  
115In(n,n')     0.02     10.950187        0.009855  
115In(n,g)      0.10      4.545667        0.035911  


In [82]:
(countDF,countOrder,countTime)=optimal_count_plan(simFoilParams, handleTime=60, detR=detR, det2FoilDist=det2FoilDist, \
                                                  background=background, units='Bq', toMinute = True)
print 'Total count time = {:2e} hrs\n'.format(countTime/3600)
print 'The order of counting is: {}\n'.format(countOrder)
for ind in countDF.index:
    print "{:15s} {:4.0f} min".format(ind,countDF.at[ind,"countTime"]/60)

Total count time = 2.110333e+02 hrs

The order of counting is: (u'In', u'NiP')

115In(n,n')      102 min
115In(n,g)       102 min
58Ni(n,p)       12560 min


# The neutron intensity is too low to run foil activation for this experiment.  

#### Option 1) 
3 channels are lost; 3 have terrible statistics. <br/>

Total count time = 2.695000e+01 hrs

The order of counting is: (u'In', u'AlA', u'NiP')

115In(n,n')       53 min  <br/>
115In(n,g)        53 min <br/>
58Ni(n,p)        477 min <br/>
27Al(n,a)       1087 min <br/>

#### Option 2)
4 channels are lost; 1 has terrible statistics. Can count forever to get 2 channels with decent statistics and one with 10% stats. <br/>

Total count time = 9.038333e+01 hrs

The order of counting is: (u'In', u'NiP')

115In(n,n')      102 min <br/>
115In(n,g)       102 min <br/>
58Ni(n,p)       12560 min <br/>